In [15]:
import re
from collections import Counter
import numpy as np
import pandas as pd

In [16]:
import re
import numpy as np
import pandas as pd
from collections import Counter

_IUPAC = {
    'R': ('A', 'G'), 'Y': ('C', 'T'), 'S': ('G', 'C'),
    'W': ('A', 'T'), 'K': ('G', 'T'), 'M': ('A', 'C')
}

def normalize_snp_alleles(genotype):
    """
    Normalize any diploid SNP genotype to a tuple of two alleles.
    Accepts any separator or compact form:
        AA, A/A, A-T, A:T, A;T, A|T, "A;A"
    Returns:
        tuple('A','T') or None if not interpretable as diploid SNP
    """
    if pd.isna(genotype):
        return None
    s = str(genotype).upper()
    # Extract only valid nucleotide alleles, ignore separators
    alleles = re.findall(r'[ACGT]', s)
    if len(alleles) == 2:
        return tuple(alleles)
    return None


def detect_and_convert_markers_to_012(markers_df, sample_size=1000):
    """
    Detect marker format and convert SNP-like columns to 0/1/2 encoding when possible.
    - Keeps first column as ID.
    - Prints detected type and explanation.
    - Sets global `markers_type` (string).
    - Returns (converted_df, markers_type).
    """

    if not isinstance(markers_df, pd.DataFrame):
        raise TypeError("markers_df must be a pandas DataFrame")

    if markers_df.shape[1] == 0:
        raise ValueError("Empty dataframe (no columns).")

    id_col = markers_df.columns[0]
    data_cols = list(markers_df.columns[1:])
    if not data_cols:
        print("No marker columns found (only ID column). Nothing to convert.")
        globals()['markers_type'] = None
        return markers_df.copy(), None

    # ---- sampling helper ----
    def sample_values():
        stacked = markers_df[data_cols].astype(str).stack()
        if len(stacked) == 0:
            return []
        if len(stacked) <= sample_size:
            return stacked.tolist()
        return stacked.sample(n=sample_size, random_state=1).tolist()

    samples = sample_values()

    # ---- regex patterns ----
    re_numeric_012 = re.compile(r'^[0-2]$')
    re_numeric_01 = re.compile(r'^[01]$')
    re_vcf_gt = re.compile(r'^[01][\/|][01]$')   # 0/0 0/1 1/1
    re_slash_numeric = re.compile(r'^\s*\d+\s*\/\s*\d+\s*$')
    re_iupac = re.compile(r'^[RYSWKMryswkm]$')

    # ---- detect marker types by content, not separators ----
    counts = Counter()
    for v in samples:
        s = str(v).strip()

        if s == '' or s.lower() in ('nan', 'none'):
            counts['missing'] += 1
            continue

        # Extract nucleotide alleles ignoring any separator
        alleles = re.findall(r'[ACGTacgt]', s)

        # Exactly two alleles -> diploid SNP (any separator)
        if len(alleles) == 2:
            counts['allelic_diploid'] += 1
        elif re_vcf_gt.match(s):
            counts['vcf_gt'] += 1
        elif re_slash_numeric.match(s):
            counts['microsatellite_like'] += 1
        elif re_numeric_012.match(s):
            counts['numeric_012'] += 1
        elif re_numeric_01.match(s):
            counts['numeric_01'] += 1
        elif re_iupac.match(s):
            counts['iupac'] += 1
        else:
            counts['other'] += 1

    most_common_count = counts.most_common(1)[0][1] if counts else 0

    # ---- decide overall marker type ----
    if counts['microsatellite_like'] and counts['microsatellite_like'] >= most_common_count:
        markers_type = 'microsatellites'
        explanation = "Detected numeric allele sizes (e.g. '150/152') -> microsatellites."
    elif counts['vcf_gt'] and counts['vcf_gt'] >= most_common_count:
        markers_type = 'vcf_gt'
        explanation = "Detected VCF GT style (e.g. '0/1','1/1')."
    elif counts['numeric_012'] and counts['numeric_012'] >= most_common_count:
        markers_type = 'numeric_012'
        explanation = "Detected numeric SNP genotypes (0/1/2)."
    elif counts['allelic_diploid'] and counts['allelic_diploid'] >= most_common_count:
        markers_type = 'allelic_diploid'
        explanation = "Detected diploid SNP genotypes with arbitrary separators."
    elif counts['numeric_01'] and counts['numeric_01'] >= most_common_count:
        markers_type = 'presence_absence'
        explanation = "Detected presence/absence data (0/1)."
    elif counts['iupac'] and counts['iupac'] >= most_common_count:
        markers_type = 'iupac'
        explanation = "Detected IUPAC ambiguity codes."
    else:
        markers_type = 'unknown'
        explanation = "Could not confidently determine marker format."

    pretty = {
        'microsatellites': 'Microsatellites',
        'vcf_gt': 'VCF GT (0/1)',
        'numeric_012': 'Numeric SNPs (0/1/2)',
        'allelic_diploid': 'Allelic SNPs (any separator)',
        'presence_absence': 'Presence/Absence (0/1)',
        'iupac': 'IUPAC codes',
        'unknown': 'Unknown'
    }

    print(f"Markers type detected: {pretty.get(markers_type, markers_type)}")
    print("Details:", explanation)
    globals()['markers_type'] = markers_type

    # ---- formats that should not be auto-converted ----
    if markers_type in ('microsatellites', 'unknown', 'presence_absence'):
        print("No automatic conversion performed for this marker type.")
        return markers_df.copy(), markers_type

    # ---- prepare output ----
    df_out = markers_df.copy()

    # ---- column-wise conversion ----
    for col in data_cols:
        col_series = markers_df[col]
        non_null = col_series.dropna().astype(str).str.strip()

        if non_null.empty:
            df_out[col] = np.nan
            continue

        # ---- VCF GT ----
        if non_null.str.match(re_vcf_gt).all():
            def map_vcf(x):
                if pd.isna(x):
                    return np.nan
                a, b = re.split(r'[\/|]', str(x))
                return int(a) + int(b)
            df_out[col] = col_series.map(map_vcf).astype(float)
            continue

        # ---- numeric 0/1/2 ----
        if non_null.str.fullmatch(r'[0-2]').all():
            df_out[col] = pd.to_numeric(col_series, errors='coerce').astype(float)
            continue

        # ---- diploid SNPs with any separator ----
        allele_list = []
        for g in non_null:
            norm = normalize_snp_alleles(g)
            if norm:
                allele_list.extend(norm)

        if allele_list:
            allele_counts = Counter(allele_list)
            ref = allele_counts.most_common(1)[0][0]
            alt = allele_counts.most_common(2)[1][0] if len(allele_counts) > 1 else ref

            def map_snp(g):
                if pd.isna(g):
                    return np.nan
                norm = normalize_snp_alleles(g)
                if not norm:
                    return np.nan
                return int(norm[0] == alt) + int(norm[1] == alt)

            df_out[col] = col_series.map(map_snp).astype(float)
            #print(f"Column '{col}': detected SNPs; REF='{ref}', ALT='{alt}'")
            continue

        # ---- IUPAC ----
        if non_null.str.fullmatch(r'[RYSWKMryswkm]').all():
            df_out[col] = col_series.map(
                lambda x: 1.0 if str(x).upper() in _IUPAC else np.nan
            ).astype(float)
            continue

        # ---- fallback ----
        coerced = pd.to_numeric(col_series, errors='coerce')
        coerced = coerced.where(coerced.isin([0, 1, 2]), np.nan)
        df_out[col] = coerced.astype(float)
        print(f"Column '{col}': fallback numeric coercion.")

    print("Conversion complete. Global markers_type:", markers_type)
    return df_out, markers_type


In [17]:
def detect_and_convert_markers_to_012(markers_df, sample_size=1000):
    """
    Detect marker format and convert SNP-like columns to 0/1/2 encoding when possible.

    Behavior
    --------
    - Keeps first column as ID.
    - Detects marker type from data content.
    - Prints detected marker type and explanation.
    - Sets global variable `markers_type`.
    - RETURNS ONLY the converted DataFrame.
    """

    if not isinstance(markers_df, pd.DataFrame):
        raise TypeError("markers_df must be a pandas DataFrame")

    if markers_df.shape[1] == 0:
        raise ValueError("Empty dataframe (no columns).")

    id_col = markers_df.columns[0]
    data_cols = list(markers_df.columns[1:])

    if not data_cols:
        print("No marker columns found (only ID column). Nothing to convert.")
        globals()['markers_type'] = None
        return markers_df.copy()

    # ---- sampling helper ----
    def sample_values():
        stacked = markers_df[data_cols].astype(str).stack()
        if stacked.empty:
            return []
        if len(stacked) <= sample_size:
            return stacked.tolist()
        return stacked.sample(n=sample_size, random_state=1).tolist()

    samples = sample_values()

    # ---- regex patterns ----
    re_numeric_012 = re.compile(r'^[0-2]$')
    re_numeric_01 = re.compile(r'^[01]$')
    re_vcf_gt = re.compile(r'^[01][\/|][01]$')
    re_slash_numeric = re.compile(r'^\s*\d+\s*\/\s*\d+\s*$')
    re_iupac = re.compile(r'^[RYSWKMryswkm]$')

    # ---- detect marker types ----
    counts = Counter()

    for v in samples:
        s = str(v).strip()

        if s == '' or s.lower() in ('nan', 'none'):
            counts['missing'] += 1
            continue

        alleles = re.findall(r'[ACGTacgt]', s)

        if len(alleles) == 2:
            counts['allelic_diploid'] += 1
        elif re_vcf_gt.match(s):
            counts['vcf_gt'] += 1
        elif re_slash_numeric.match(s):
            counts['microsatellite_like'] += 1
        elif re_numeric_012.match(s):
            counts['numeric_012'] += 1
        elif re_numeric_01.match(s):
            counts['numeric_01'] += 1
        elif re_iupac.match(s):
            counts['iupac'] += 1
        else:
            counts['other'] += 1

    most_common_count = counts.most_common(1)[0][1] if counts else 0

    # ---- decide marker type ----
    if counts['microsatellite_like'] >= most_common_count and counts['microsatellite_like'] > 0:
        markers_type = 'microsatellites'
        explanation = "Detected numeric allele sizes (e.g. '150/152')."
    elif counts['vcf_gt'] >= most_common_count and counts['vcf_gt'] > 0:
        markers_type = 'vcf_gt'
        explanation = "Detected VCF GT format (e.g. '0/1')."
    elif counts['numeric_012'] >= most_common_count and counts['numeric_012'] > 0:
        markers_type = 'numeric_012'
        explanation = "Detected numeric SNP encoding (0/1/2)."
    elif counts['allelic_diploid'] >= most_common_count and counts['allelic_diploid'] > 0:
        markers_type = 'allelic_diploid'
        explanation = "Detected diploid SNPs with arbitrary separators."
    elif counts['numeric_01'] >= most_common_count and counts['numeric_01'] > 0:
        markers_type = 'presence_absence'
        explanation = "Detected presence/absence markers (0/1)."
    elif counts['iupac'] >= most_common_count and counts['iupac'] > 0:
        markers_type = 'iupac'
        explanation = "Detected IUPAC ambiguity codes."
    else:
        markers_type = 'unknown'
        explanation = "Could not confidently determine marker format."

    print(f"Markers type detected: {markers_type}")
    print("Details:", explanation)

    globals()['markers_type'] = markers_type

    # ---- formats that should not be auto-converted ----
    if markers_type in ('microsatellites', 'presence_absence', 'unknown'):
        print("No automatic conversion performed for this marker type.")
        return markers_df.copy()

    # ---- prepare output ----
    df_out = markers_df.copy()

    # ---- column-wise conversion ----
    for col in data_cols:
        col_series = markers_df[col]
        non_null = col_series.dropna().astype(str).str.strip()

        if non_null.empty:
            df_out[col] = np.nan
            continue

        # ---- VCF GT ----
        if non_null.str.match(re_vcf_gt).all():
            def map_vcf(x):
                if pd.isna(x):
                    return np.nan
                a, b = re.split(r'[\/|]', str(x))
                return int(a) + int(b)

            df_out[col] = col_series.map(map_vcf).astype(float)
            continue

        # ---- numeric 0/1/2 ----
        if non_null.str.fullmatch(r'[0-2]').all():
            df_out[col] = pd.to_numeric(col_series, errors='coerce').astype(float)
            continue

        # ---- diploid SNPs ----
        allele_list = []
        for g in non_null:
            norm = normalize_snp_alleles(g)
            if norm:
                allele_list.extend(norm)

        if allele_list:
            allele_counts = Counter(allele_list)
            ref = allele_counts.most_common(1)[0][0]
            alt = allele_counts.most_common(2)[1][0] if len(allele_counts) > 1 else ref

            def map_snp(g):
                if pd.isna(g):
                    return np.nan
                norm = normalize_snp_alleles(g)
                if not norm:
                    return np.nan
                return int(norm[0] == alt) + int(norm[1] == alt)

            df_out[col] = col_series.map(map_snp).astype(float)
            continue

        # ---- IUPAC ----
        if non_null.str.fullmatch(r'[RYSWKMryswkm]').all():
            df_out[col] = col_series.map(
                lambda x: 1.0 if str(x).upper() in _IUPAC else np.nan
            ).astype(float)
            continue

        # ---- fallback ----
        coerced = pd.to_numeric(col_series, errors='coerce')
        coerced = coerced.where(coerced.isin([0, 1, 2]), np.nan)
        df_out[col] = coerced.astype(float)
        print(f"Column '{col}': fallback numeric coercion.")

    print("Conversion complete.")
    print("Global markers_type:", markers_type)

    return df_out


# Data load

In [18]:
micros = pd.read_csv('./data_type_test/microsatellites.csv')
micros

,ID,M1,M2,M3
0,Ind1,150/152,198/204,100/102
1,Ind2,148/150,200/200,102/104
2,Ind3,150/150,198/198,100/104
3,Ind4,152/152,204/204,104/104


In [19]:
snps_012 = pd.read_csv('./data_type_test/snps_012.csv')
snps_012

,ID,SNP1,SNP2,SNP3
0,Ind1,0,1,2
1,Ind2,1,1,0
2,Ind3,2,0,1
3,Ind4,1,2,2


In [20]:
snps_allelic = pd.read_csv('./data_type_test/snps_allelic_twoletter.csv')
snps_allelic

,ID,SNP1,SNP2,SNP3
0,Ind1,AA,AG,GG
1,Ind2,AG,GG,AA
2,Ind3,GG,AG,AG
3,Ind4,AA,AA,GG


In [21]:
snps_vcf = pd.read_csv('./data_type_test/snps_vcf.csv')
snps_vcf

,ID,SNP1,SNP2,SNP3
0,Ind1,0/0,0/1,1/1
1,Ind2,0/1,0/1,0/0
2,Ind3,1/1,0/0,0/1
3,Ind4,0/0,1/1,1/1


In [22]:
snp_slash_format = pd.read_csv('./data_type_test/snp_slash_format.csv')
snp_slash_format

,sample,SNP1,SNP2,SNP3
0,ind1,A/A,C/C,G/G
1,ind2,A/G,C/T,G/A
2,ind3,G/G,T/T,A/A


In [23]:
snp_variable_format = pd.read_csv('./data_type_test/snps_semicolon_format.csv')

In [24]:
snp_variable_format = snp_variable_format.drop('pop', axis=1)
snp_variable_format.head(3)

,Unnamed: 0,100372750-33-A/G,100352026-35-G/A,100398603-66-T/A,100282003-7-C/A,100268131-40-A/G,100346786-19-C/T,100385025-37-A/G,100454812-47-G/T,100249878-21-T/C,...,100472466-7-G/A,100269708-12-A/G,100247673-6-C/T,100400729-21-T/A,100249247-8-C/A,100493546-6-A/T,100250134-41-G/A,100247596-22-T/C,100401377-8-T/C,100318416-9-C/G
0,Cly 1,G;G,G;G,A;A,A;A,A;A,C;C,G;G,G;G,T;T,...,G;G,G;G,T;T,A;A,A;A,T;T,G;G,T;T,C;C,C;C
1,J-9,A;A,A;A,T;T,C;C,G;G,T;T,A;A,T;T,T;T,...,A;A,G;G,C;C,T;T,C;C,A;A,A;A,T;T,T;T,G;G
2,H-29,A;A,A;A,T;T,C;C,G;G,T;T,A;A,T;T,T;T,...,A;A,G;G,C;C,T;T,C;C,A;A,A;A,T;T,T;T,G;G


# Tests

In [25]:
detect_and_convert_markers_to_012(micros)

Markers type detected: microsatellites
Details: Detected numeric allele sizes (e.g. '150/152').
No automatic conversion performed for this marker type.


,ID,M1,M2,M3
0,Ind1,150/152,198/204,100/102
1,Ind2,148/150,200/200,102/104
2,Ind3,150/150,198/198,100/104
3,Ind4,152/152,204/204,104/104


In [26]:
detect_and_convert_markers_to_012(snps_012)

Markers type detected: numeric_012
Details: Detected numeric SNP encoding (0/1/2).
Conversion complete.
Global markers_type: numeric_012


,ID,SNP1,SNP2,SNP3
0,Ind1,0.0,1.0,2.0
1,Ind2,1.0,1.0,0.0
2,Ind3,2.0,0.0,1.0
3,Ind4,1.0,2.0,2.0


In [27]:
detect_and_convert_markers_to_012(snps_allelic)

Markers type detected: allelic_diploid
Details: Detected diploid SNPs with arbitrary separators.
Conversion complete.
Global markers_type: allelic_diploid


,ID,SNP1,SNP2,SNP3
0,Ind1,0.0,1.0,0.0
1,Ind2,1.0,2.0,2.0
2,Ind3,2.0,1.0,1.0
3,Ind4,0.0,0.0,0.0


In [28]:
detect_and_convert_markers_to_012(snps_vcf)

Markers type detected: vcf_gt
Details: Detected VCF GT format (e.g. '0/1').
Conversion complete.
Global markers_type: vcf_gt


,ID,SNP1,SNP2,SNP3
0,Ind1,0.0,1.0,2.0
1,Ind2,1.0,1.0,0.0
2,Ind3,2.0,0.0,1.0
3,Ind4,0.0,2.0,2.0


In [29]:
detect_and_convert_markers_to_012(snp_slash_format)

Markers type detected: allelic_diploid
Details: Detected diploid SNPs with arbitrary separators.
Conversion complete.
Global markers_type: allelic_diploid


,sample,SNP1,SNP2,SNP3
0,ind1,0.0,0.0,0.0
1,ind2,1.0,1.0,1.0
2,ind3,2.0,2.0,2.0


In [30]:
detect_and_convert_markers_to_012(snp_variable_format)

Markers type detected: allelic_diploid
Details: Detected diploid SNPs with arbitrary separators.
Conversion complete.
Global markers_type: allelic_diploid


,Unnamed: 0,100372750-33-A/G,100352026-35-G/A,100398603-66-T/A,100282003-7-C/A,100268131-40-A/G,100346786-19-C/T,100385025-37-A/G,100454812-47-G/T,100249878-21-T/C,...,100472466-7-G/A,100269708-12-A/G,100247673-6-C/T,100400729-21-T/A,100249247-8-C/A,100493546-6-A/T,100250134-41-G/A,100247596-22-T/C,100401377-8-T/C,100318416-9-C/G
0,Cly 1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,...,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0
1,J-9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,H-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,D-7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,I-5B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,Vin 34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,CVLC-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,E-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
snps_wild = pd.read_csv('./chili/wild/snps_merged.csv')
snps_wild = detect_and_convert_markers_to_012(snps_wild)


Markers type detected: allelic_diploid
Details: Detected diploid SNPs with arbitrary separators.
Conversion complete.
Global markers_type: allelic_diploid


In [34]:
def filter_variable_snps(df, id_col="sample"):
    snp_cols = df.columns.drop(id_col)
    keep = df[snp_cols].nunique(dropna=True) > 1
    return df[[id_col] + list(snp_cols[keep])]

In [39]:
import numpy as np
import pandas as pd

def filter_snps_by_maf(
    df,
    id_col="sample",
    maf_threshold=0.05
):
    """
    Filter SNPs by Minor Allele Frequency (MAF).

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with ID column + SNPs encoded as 0/1/2.
    id_col : str
        Name of the ID/sample column.
    maf_threshold : float
        Minimum MAF required to keep a SNP (e.g. 0.05).

    Returns
    -------
    pd.DataFrame
        Filtered DataFrame containing only SNPs with MAF >= threshold.
    """

    if id_col not in df.columns:
        raise KeyError(f"ID column '{id_col}' not found.")

    snp_cols = df.columns.drop(id_col)

    if len(snp_cols) == 0:
        raise ValueError("No SNP columns found.")

    mafs = {}

    for col in snp_cols:
        g = df[col].dropna()

        if g.empty:
            mafs[col] = np.nan
            continue

        # ALT allele count
        alt_count = g.sum()

        # Total alleles
        total_alleles = 2 * len(g)

        freq = alt_count / total_alleles
        maf = min(freq, 1 - freq)

        mafs[col] = maf

    maf_series = pd.Series(mafs)

    # Keep SNPs above threshold
    keep_snps = maf_series[maf_series >= maf_threshold].index.tolist()

    if not keep_snps:
        raise ValueError(
            f"No SNPs passed MAF threshold ({maf_threshold})."
        )

    return df[[id_col] + keep_snps]


In [40]:
snps_wild = filter_snps_by_maf(snps_wild)

In [41]:
snps_wild

,sample,100426978-22-C/T,100511668-55-C/T,100401792-27-G/A,100268448-39-C/T,100171344-48-G/C,100400274-7-T/C,100444323-13-C/G,100417228-31-T/G,100318504-28-T/C,...,100172292-58-C/A,100330897-41-T/C,100375367-19-C/T,100173191-30-T/C,100168929-8-A/G,100376870-21-A/G,100288190-51-G/A,100167474-9-T/C,100174868-17-A/G,100442732-31-A/G
0,B-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0
1,B-7,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,B-9B,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
3,C-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0
4,C-2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
5,C-alfa,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
6,C-G,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
7,C-H,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8,C-K,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D-5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [42]:
snps_wild.to_csv('./chili/wild/snps_merged_012.csv', index=None)

# Errors load

In [13]:
mixed_types_error = pd.read_csv('./data_type_test/mixed_types_error.csv')
mixed_types_error

,ID,SNP1,SNP2
0,A,0,150/152
1,B,AA,0/1
2,C,1,GG
3,D,AG,1


In [14]:
presence_absence = pd.read_csv('./data_type_test/presence_absence.csv')
presence_absence

,ID,M1,M2,M3
0,Ind1,1.0,0,1.0
1,Ind2,0.0,0,1.0
2,Ind3,1.0,1,NaN
3,Ind4,NaN,1,0.0


In [15]:
malformed_entries = pd.read_csv('./data_type_test/malformed_entries.csv')
malformed_entries

,ID,L1,L2
0,s1,A/A,0/0
1,s2,A;G,0|1
2,s3,??,1/1
3,s4,A/G,not_known


In [16]:
microsatellite_malformed = pd.read_csv('./data_type_test/microsatellite_malformed.csv')
microsatellite_malformed

,ID,M1,M2
0,i1,150/152,200/200
1,i2,148/150,198/198
2,i3,150/A,-
3,i4,152/152,204/204


# Errors Test

In [17]:
detect_and_convert_markers_to_012(mixed_types_error)

Markers type detected: Numeric SNPs (0/1/2)
Details: Detected numeric genotypes (0/1/2).
Column 'SNP1': coerced to numeric 0/1/2 where possible.
Column 'SNP2': coerced to numeric 0/1/2 where possible.
Conversion complete where possible. Global markers_type: numeric_012


(  ID  SNP1  SNP2
 0  A   0.0   NaN
 1  B   NaN   NaN
 2  C   1.0   NaN
 3  D   NaN   1.0,
 'numeric_012')

In [18]:
detect_and_convert_markers_to_012(presence_absence)

Markers type detected: Unknown
Details: Could not confidently determine marker format.
No automatic conversion performed for microsatellites/unknown formats.


(     ID   M1  M2   M3
 0  Ind1  1.0   0  1.0
 1  Ind2  0.0   0  1.0
 2  Ind3  1.0   1  NaN
 3  Ind4  NaN   1  0.0,
 'unknown')

In [19]:
detect_and_convert_markers_to_012(malformed_entries)

Markers type detected: Unknown
Details: Could not confidently determine marker format.
No automatic conversion performed for microsatellites/unknown formats.


(   ID   L1         L2
 0  s1  A/A        0/0
 1  s2  A;G        0|1
 2  s3   ??        1/1
 3  s4  A/G  not_known,
 'unknown')

In [20]:
detect_and_convert_markers_to_012(microsatellite_malformed)

Markers type detected: Microsatellites
Details: Detected numeric-slash entries (e.g. '150/152') -> microsatellites.
No automatic conversion performed for microsatellites/unknown formats.


(   ID       M1       M2
 0  i1  150/152  200/200
 1  i2  148/150  198/198
 2  i3    150/A        -
 3  i4  152/152  204/204,
 'microsatellites')